In [ ]:
import requests
import json

In [3]:
URL = 'http://192.168.32.1:9094/api/projects/1/export'
TOKEN = 'Token 757c19744cc9f63c6fc1df4ac30884a90d47b151'

In [13]:
requests.get(url = f'{URL}?ids[]=555',
             headers = {'Authorization': TOKEN})

<Response [200]>

In [20]:
START = 637
END = 993

In [33]:
ENTITY_LIST = {}

In [34]:
def convert_sample(sample):
    sent = sample['text'][4:]
    # if 'label' in sample:
    #     print(sample)
    #     return 
    label = sample['label']

    label_tuple = []
    for item in label:
        if '<i>' in item['text']:
            assert len(item['labels']) == 1, f'Got overlapse labels for intent!'
            intent = item['labels'][0][7:]
        else:
            label_tuple.append(item)
    label_tuple.sort(key=lambda x : x['start'])
    last_e_index = 0
    cv_sent = ''
    if len(label_tuple) == 0:
        return intent, sent
    
    for item in label_tuple:
        # print(item)
        assert len(item['labels']) == 1, f'Got overlapse labels for entity!'
        s_index = item['start']
        e_index = item['end']
        text = item['text']
        # print(item['labels'][0])
        try:
            entity, role = item['labels'][0].split('|')
            if entity in ENTITY_LIST.keys():
                if role not in ENTITY_LIST[entity]: ENTITY_LIST[entity].append(role)
            else:
                ENTITY_LIST[entity] = [role]
        except Exception:
            entity, role = item['labels'][0], None
            if entity in ENTITY_LIST.keys():
                ENTITY_LIST[entity] = []

        if entity == 'intent': 
            print(sample)

        cv_sent += sent[last_e_index : s_index-4]

        cv_sent += '[' + text + ']{"entity": "' + entity + '"'
        cv_sent += ', "role": "' + role + '"}' if role is not None else '}'

        last_e_index = e_index-4

    return intent, cv_sent

In [39]:
export_data = {}
failed_id = []

for id in range(START, END + 1):
    response = requests.get(url = f'{URL}?ids[]={id}',
                            headers = {'Authorization': TOKEN})
    try:
        response = response.json()[0]
        # print(response)
        text = response["data"]["text"]
        label = [item["value"] for item in response["annotations"][-1]["result"]]
        
        intent, cv_sent = convert_sample(
                {
                    "id": id,
                    "text": text,
                    "label": label
                }
            )
        
        if intent not in export_data.keys():
            export_data[intent] = []
        else:
            export_data[intent].append(cv_sent)

        # print(export_data[-1])
        
        
    except Exception as e:
        print(e.args, response.status_code)
        failed_id.append(id)
    
print(failed_id)

('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list index out of range',) 200
('list ind

In [40]:
len(export_data)

10

In [65]:
nlu_format = {
    "version": "3.1",
    "nlu": [
        
    ]
}

for intent in export_data.keys():
    nlu_format["nlu"].append({
        "intent": intent,
        "examples": "- " + "\n- ".join(export_data.get(intent))
    })

In [69]:
print(json.dumps(nlu_format, ensure_ascii=False))

{"version": "3.1", "nlu": [{"intent": "draw_line", "examples": "- Cho bức tranh của bạn một chút phá cách bằng cách vẽ một [đường thẳng]{\"entity\": \"value\", \"role\": \"shape\"} nghiêng tại góc ( [75]{\"entity\": \"position\", \"role\": \"source_x\"} , [100]{\"entity\": \"position\", \"role\": \"source_y\"} ) và [kết thúc]{\"entity\": \"aspect\"} tại ( [175]{\"entity\": \"position\", \"role\": \"destination_x\"} , [250]{\"entity\": \"position\", \"role\": \"destination_y\"} ) , màu [đỏ]{\"entity\": \"value\", \"role\": \"color\"}\n- Tô điểm bức tranh với một [đường thẳng]{\"entity\": \"value\", \"role\": \"shape\"} nằm [ngang]{\"entity\": \"value\", \"role\": \"orientation\"} tại [vị trí]{\"entity\": \"aspect\"} ( [300]{\"entity\": \"position\", \"role\": \"source_x\"} , [50]{\"entity\": \"position\", \"role\": \"source_y\"} ) , [dài]{\"entity\": \"aspect\"} [120]{\"entity\": \"value\", \"role\": \"length\"} pixel , màu [cam]{\"entity\": \"value\", \"role\": \"color\"}\n- Đưa ra sự 

In [77]:
import yaml

print(yaml.dump(nlu_format, allow_unicode=True, width=100000, sort_keys=False))

version: '3.1'
nlu:
- intent: draw_line
  examples: '- Cho bức tranh của bạn một chút phá cách bằng cách vẽ một [đường thẳng]{"entity": "value", "role": "shape"} nghiêng tại góc ( [75]{"entity": "position", "role": "source_x"} , [100]{"entity": "position", "role": "source_y"} ) và [kết thúc]{"entity": "aspect"} tại ( [175]{"entity": "position", "role": "destination_x"} , [250]{"entity": "position", "role": "destination_y"} ) , màu [đỏ]{"entity": "value", "role": "color"}

    - Tô điểm bức tranh với một [đường thẳng]{"entity": "value", "role": "shape"} nằm [ngang]{"entity": "value", "role": "orientation"} tại [vị trí]{"entity": "aspect"} ( [300]{"entity": "position", "role": "source_x"} , [50]{"entity": "position", "role": "source_y"} ) , [dài]{"entity": "aspect"} [120]{"entity": "value", "role": "length"} pixel , màu [cam]{"entity": "value", "role": "color"}

    - Đưa ra sự ấn tượng với một [đường thẳng]{"entity": "value", "role": "shape"} màu [hồng]{"entity": "value", "role": "color